In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.sparse import coo_matrix
from scipy import sparse

## load_data

In [3]:
import struct
import matplotlib.pyplot as plt

# 训练集文件
train_images_idx3_ubyte_file = '../data/common/train-images-idx3-ubyte'
# 训练集标签文件
train_labels_idx1_ubyte_file = '../data/common/train-labels-idx1-ubyte'

# 测试集文件
test_images_idx3_ubyte_file = '../data/common/t10k-images-idx3-ubyte'
# 测试集标签文件
test_labels_idx1_ubyte_file = '../data/common/t10k-labels-idx1-ubyte'
def decode_idx3_ubyte(idx3_ubyte_file):
    """
    解析idx3文件的通用函数
    :param idx3_ubyte_file: idx3文件路径
    :return: 数据集
    """
    # 读取二进制数据
    bin_data = open(idx3_ubyte_file, 'rb').read()

    # 解析文件头信息，依次为魔数、图片数量、每张图片高、每张图片宽
    offset = 0
    fmt_header = '>iiii'
    magic_number, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, bin_data, offset)
    print('魔数:%d, 图片数量: %d张, 图片大小: %d*%d' % (magic_number, num_images, num_rows, num_cols))

    # 解析数据集
    image_size = num_rows * num_cols
    offset += struct.calcsize(fmt_header)
    fmt_image = '>' + str(image_size) + 'B'
    images = np.empty((num_images, num_rows, num_cols))
    for i in range(num_images):
        if (i + 1) % 10000 == 0:
            print('已解析 %d' % (i + 1) + '张')
        images[i] = np.array(struct.unpack_from(fmt_image, bin_data, offset)).reshape((num_rows, num_cols))
        offset += struct.calcsize(fmt_image)
    return images


def decode_idx1_ubyte(idx1_ubyte_file):
    """
    解析idx1文件的通用函数
    :param idx1_ubyte_file: idx1文件路径
    :return: 数据集
    """
    # 读取二进制数据
    bin_data = open(idx1_ubyte_file, 'rb').read()

    # 解析文件头信息，依次为魔数和标签数
    offset = 0
    fmt_header = '>ii'
    magic_number, num_images = struct.unpack_from(fmt_header, bin_data, offset)
    print('魔数:%d, 图片数量: %d张' % (magic_number, num_images))

    # 解析数据集
    offset += struct.calcsize(fmt_header)
    fmt_image = '>B'
    labels = np.empty(num_images)
    for i in range(num_images):
        if (i + 1) % 10000 == 0:
            print('已解析 %d' % (i + 1) + '张')
        labels[i] = struct.unpack_from(fmt_image, bin_data, offset)[0]
        offset += struct.calcsize(fmt_image)
    return labels
train_image=decode_idx3_ubyte(train_images_idx3_ubyte_file)
test_image=decode_idx3_ubyte(test_images_idx3_ubyte_file)
train_label=decode_idx1_ubyte(train_labels_idx1_ubyte_file)
test_label=decode_idx1_ubyte(test_labels_idx1_ubyte_file)

魔数:2051, 图片数量: 60000张, 图片大小: 28*28
已解析 10000张
已解析 20000张
已解析 30000张
已解析 40000张
已解析 50000张
已解析 60000张
魔数:2051, 图片数量: 10000张, 图片大小: 28*28
已解析 10000张
魔数:2049, 图片数量: 60000张
已解析 10000张
已解析 20000张
已解析 30000张
已解析 40000张
已解析 50000张
已解析 60000张
魔数:2049, 图片数量: 10000张
已解析 10000张


In [5]:
train_image.shape

(60000, 28, 28)

In [29]:
y=np.array(np.arange(10))
cols=y
rows=np.array([0,3,1,2,2,1,0,3,2,1])
data=np.array([1,1,1,1,1,1,1,1,1,1])
coo_mat = sparse.coo_matrix((data, (rows, cols)), shape=(4, 10)).toarray()

In [31]:
coo_mat

array([[1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0]])

In [91]:
#function [f,g] = softmax_regression(theta, X,y)
#   %
#   % Arguments:
#   %   theta - A vector containing the parameter values to optimize.
#   %       In minFunc, theta is reshaped to a long vector.  So we need to
#   %       resize it to an n-by-(num_classes-1) matrix.
#   %       Recall that we assume theta(:,num_classes) = 0.
#   %
#   %   X - The examples stored in a matrix.  
#   %       X(i,j) is the i'th coordinate of the j'th example.
#   %   y - The label for each example.  y(j) is the j'th example's label.
#   %
def softmax_reg(theta,X,y):
    m=X.shape[1];
    n=X.shape[0];
  #   % initialize objective value and gradient.

#   % theta is a vector;  need to reshape to n x num_classes.
    #theta=reshape(theta, n, []);theta=theta.reshape()
    num_classes=theta.shape[0]
  #% initialize objective value and gradient.
    f = 0;
    g = np.zeros(shape=theta.shape)
    re=np.dot(theta,X)
    print(re)
    re1=np.exp(re)
    re2=np.sum(re1,axis=0)
    re3=re1/re2
    re4=np.log(re3)
    cols=np.array(np.arange(m))
    rows=y
    data=np.array([1]*m)
    coo_mat = sparse.coo_matrix((data, (rows, cols)), shape=(num_classes, m)).toarray()
    re5=coo_mat*re4
    f=-np.sum(re5)
    g=np.dot((coo_mat-re3),X.T)
    return f,g
def cost_func(theta,X,y):
    m=X.shape[1];
    n=X.shape[0];
    num_classes=theta.shape[0]
    f = 0;
    g = np.zeros(shape=theta.shape)
    re=np.dot(theta,X)
    re1=np.exp(re)
    re2=np.sum(re1,axis=0)
    re3=re1/re2
    re4=np.log(re3)
    cols=np.array(np.arange(m))
    rows=y
    data=np.array([1]*m)
    coo_mat = sparse.coo_matrix((data, (rows, cols)), shape=(num_classes, m)).toarray()
    re5=coo_mat*re4
    f=-np.sum(re5)
    return f
def gredient_check(g,theta,k,X,y):
    epsilon = 1e-4
    checked=[]
    n=theta.shape[1]
    for i in range(n):
        theta1=theta
        theta2=theta
        theta1[k][i]-=epsilon
        theta2[k][i]+=epsilon
        print("cost1=",cost_func(theta1,X,y))
        print("cost2=",cost_func(theta2,X,y))
        check=(cost_func(theta2,X,y)-cost_func(theta1,X,y))/2*epsilon
        checked.append(check)
        print("gredient of %d is"%i,g[k][i])
        print("checked_gredient of %d is"%i,check)
    #   %
#   % TODO:  Compute the softmax objective function and gradient using vectorized code.
#   %        Store the objective function value in 'f', and the gradient in 'g'.
#   %        Before returning g, make sure you form it back into a vector with g=g(:);
#   %
# %%% YOUR CODE HERE %%%
  
#     g=g(:); % make gradient a vector for minFunc

## ex_softmax_reg

In [ ]:
train_X=np.empty((784,train_num))
test_X=np.empty((784,test_num))
for i in range(train_num):
    train_X[:,i]=train_images[i].reshape(784)
for i in range(test_num):
    test_X[:,i]=test_images[i].reshape(784)
cols1=train_X.shape[1]
append1=np.ones(shape=(1,cols1))
cols2=test_X.shape[1]
append2=np.ones(shape=(1,cols2))
# % Include a row of 1s as an additional intercept feature.
# % Add row of 1s to the dataset to act as an intercept term.
train_X = np.insert(train_X,0,values=append1)
test_X = np.insert(test_X,0,values=append2)
train_y=train_labels
test_y=test_labels
#% Training set dimensions
m=train_X.shape[1]
n=train_X.shape[0]
theta = init_weights(k,n);
def ex_softmax_reg(theta,X,y,lr,epoches):
    res_theta=theta
    costmin=10000
    for i in range(epoches):
        f,g=softmax_reg(theta,X,y)
        print("the cost is %f",f)
        theta-=lr*g
        if costmin>f:
            costmin=f
            res_theta=theta
    return res_theta
lr=0.05
epoches=1000
costmin=10000   
res_theta=ex_softmax_reg(theta,train_X,train_y,lr,epoches)
def classifier(theta,X):
    re=np.dot(theta,X)
    print(re)
    re1=np.exp(re)
    re2=np.sum(re1,axis=0)
    re3=re1/re2
    return re3
def init_weights(k,n_features):
    # 初始化参数
    # 参数范围(-1/sqrt(N), 1/sqrt(N))
    limit = np.sqrt(1/n_features)
    w = np.random.uniform(-limit, limit, (k,n_features))
    return w
def binary_classifier_accuracy(theta,X,y):
    predict_score=classifier(theta,X)
    predicted_y=np.argmax(predict_score,axis=0)
    lens=len(y)
    acu=np.sum(predicted_y==y)/lens
    return acu
accuracy = binary_classifier_accuracy(res_theta,train_X,train_y);
print('Training accuracy: %2.1f%\n', 100*accuracy);

#% Print out accuracy on the test set.
accuracy = binary_classifier_accuracy(res_theta,test_X,test_y);
print('Test accuracy: %2.1f%\n', 100*accuracy);


## ex_iris

In [74]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = datasets.load_iris()
X = data['data']
y = data['target']
# n:特征数，k:标签数
n = X.shape[1]
label = np.unique(y)
k = len(label)

In [75]:
def init_weights(k,n_features):
    # 初始化参数
    # 参数范围(-1/sqrt(N), 1/sqrt(N))
    limit = np.sqrt(1/n_features)
    w = np.random.uniform(-limit, limit, (k,n_features))
    return w

In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.75, test_size=0.15, random_state=1)
X_train=X_train.T
X_train=np.insert(X_train, 0, 1, axis=0)  # 添加一行
X_test=X_test.T
X_test=np.insert(X_test, 0, 1, axis=0)  # 添加一行
print(X_train.shape,X_test.shape)

(5, 112) (5, 23)


In [ ]:
m=X_train.shape[1]
n=X_train.shape[0]
theta = init_weights(k,n);
print(theta)
epoches=1000
lr=0.000001 
res_theta=ex_softmax_reg(theta,X_train,Y_train,lr,epoches)

In [16]:
data = datasets.load_iris()
X = data['data']
y = data['target']
# n:特征数，k:标签数

In [64]:
softmax=Softmax()
softmax.train(X,y)

In [63]:
import math
import pandas as pd
import numpy as np
import random
import time
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
 
class Softmax(object):
 
    def __init__(self):
        self.learning_step = 0.000001           # 学习速率
        self.max_iteration = 100000             # 最大迭代次数
        self.weight_lambda = 0.01               # 衰退权重
 
    def cal_e(self,x,l):
#         '''
#         计算博客中的公式3
#         '''
 
        theta_l = self.w[l]
        product = np.dot(theta_l,x)
 
        return math.exp(product)
 
    def cal_probability(self,x,j):
#         '''
#         计算博客中的公式2
#         '''
 
        molecule = self.cal_e(x,j)
        denominator = sum([self.cal_e(x,i) for i in range(self.k)])
 
        return molecule/denominator
 
 
    def cal_partial_derivative(self,x,y,j):
#         '''
#         计算博客中的公式1
#         '''
 
        first = int(y==j)                           # 计算示性函数
        second = self.cal_probability(x,j)          # 计算后面那个概率
 
        return -x*(first-second) + self.weight_lambda*self.w[j]
 
    def predict_(self, x):
        result = np.dot(self.w,x)
        row, column = result.shape
 
        # 找最大值所在的列
        _positon = np.argmax(result)
        m, n = divmod(_positon, column)
 
        return m
 
    def train(self, features, labels):
        self.k = len(set(labels))
 
        self.w = np.zeros((self.k,len(features[0])+1))
        time = 0
 
        while time < self.max_iteration:
            print('loop %d' % time)
            time += 1
            index = random.randint(0, len(labels) - 1)
 
            x = features[index]#选一个样本出来，features为nXm矩阵，n为样本个数，m为特征数
            y = labels[index]
 
            x = list(x)
            x.append(1.0)
            x = np.array(x)
 
            derivatives = [self.cal_partial_derivative(x,y,j) for j in range(self.k)]
 
            for j in range(self.k):
                self.w[j] -= self.learning_step * derivatives[j]
 
    def predict(self,features):
        labels = []
        for feature in features:
            x = list(feature)
            x.append(1)
 
            x = np.matrix(x)
            x = np.transpose(x)
 
            labels.append(self.predict_(x))
        return labels   
    def classifier(self,X):
        X1=X.copy()
        m=X.shape[1]
        b=np.ones(X.shape[0])
        X1=np.insert(X1,m,values=b,axis=1)
        re=np.dot(X1,self.w.T)
        print(re)
        re1=np.exp(re)
        re2=np.sum(re1,axis=1).reshape(X.shape[0],1)
        re3=re1/re2
        return re3
    def binary_classifier_accuracy(self,X,y):
        predict_score=self.classifier(X)
        predicted_y=np.argmax(predict_score,axis=1)
        lens=len(y)
        acu=np.sum(predicted_y==y)/lens
        return acu
# if __name__ == '__main__':
 
#     print('Start read data')
 
#     time_1 = time.time()
 
#     raw_data = pd.read_csv('../data/train.csv', header=0)
#     data = raw_data.values
 
#     imgs = data[0::, 1::]
#     labels = data[::, 0]
 
#     # 选取 2/3 数据作为训练集， 1/3 数据作为测试集
#     train_features, test_features, train_labels, test_labels = train_test_split(
#         imgs, labels, test_size=0.33, random_state=23323)
#     # print train_features.shape
#     # print train_features.shape
 
#     time_2 = time.time()
#     print('read data cost '+ str(time_2 - time_1)+' second')
 
#     print('Start training')
#     p = Softmax()
#     p.train(train_features, train_labels)
 
#     time_3 = time.time()
#     print('training cost '+ str(time_3 - time_2)+' second')
 
#     print('Start predicting')
#     test_predict = p.predict(test_features)
#     time_4 = time.time()
#     print('predicting cost ' + str(time_4 - time_3) +' second')
 
#     score = accuracy_score(test_labels, test_predict)
#     print("The accruacy socre is " + str(score))


In [45]:
np.random.randint(0, 5)

4